In [1]:
from ortools.linear_solver import pywraplp

In [2]:
def solve_integer_program(n, m, N, P, lambd, epsilon):
    
    # Create a solver
    solver = pywraplp.Solver.CreateSolver('CBC')  # 'CBC' is the solver used by ortools for integer programming
    #
    if not solver:
        return "Solver not available."
    #
    # Variables
    x = [solver.IntVar(0, 1, f'x{i}') for i in range(m)]  # Integer variables with lower bound 0 and upper bound 1
    zFP = [solver.IntVar(0, 1, f'z{i}') for i in range(n)]  # Auxiliary variables z_i with lower bound 0 and upper bound 1
    zFN = [solver.IntVar(0, 1, f'z{i}') for i in range(n)]
    # zFN_pre = [solver.IntVar(0, 1, f'z{i}') for i in range(n)]
    y = [solver.NumVar(0, 1, f'z{i}') for i in range(n)]
    w = [solver.NumVar(0, 1, f'z{i}') for i in range(n)]
    #
    #
    # Target function: 0^T x
    target_function = solver.Sum(x[i] * 0 for i in range(m))
    #
    # add constraints 
    n_negative = len(N) #N.shape[0]
    for i in range(n_negative):
        # Calculate N[i,:] * x (inner product)
        inner_product_N = solver.Sum(N[i][j] * x[j] for j in range(m))
        #
        # Add constraint: z_i = min(N[i,:] * x, 1)
        solver.Add(inner_product_N <= y[i] + 1)
        solver.Add(-inner_product_N <= y[i])
        solver.Add(zFP[i] <= inner_product_N)
        solver.Add(zFP[i] <= 1)
        solver.Add(zFP[i] >= 1 - (1 - y[i]))
        solver.Add(zFP[i] >= inner_product_N - y[i])
        #
        #solver.Add(zFP[i] == min([inner_product_N, 1]))
        #
    
    n_positive =  len(P) #P.shape[0]
    for i in range(n_positive): 
    
        inner_product_P = solver.Sum(1 - P[i][j] * x[j] for j in range(m))
        #
        #solver.Add(zFN[i] == (1 - max([inner_product_P, 1])))
        solver.Add(inner_product_P <= w[i] )
        solver.Add(-inner_product_P <= (1-w[i]) )
        solver.Add(zFN[i] >= inner_product_P)
        solver.Add(zFN[i] >= 0)
        solver.Add(zFN[i] <= (1 - w[i]))
        solver.Add(zFN[i] <= inner_product_P + w[i])
        #solver.Add(zFN[i] == (1 - zFN_pre[i]))
        #
        ##
        #   
        # Add constraint: sum(z_i) <= epsilon
    solver.Add(solver.Sum(zFP[i]/n for i in range(n_negative)) + solver.Sum(zFN[i]/n for i in range(n_positive)) + lambd*solver.Sum(x[h] for h in range(m)) <= epsilon)
    #   #
    #   #
    # Objective function (to be minimized): 0 - 0^T x
    # solver.Minimize(target_function)
    # Solve the problem
    
    print("here") 
    
   
    
    solutions = []
    while solver.NextSolution():
        
        print(x[i].solution_value())
        
        solution = {}
        for i in range(m):
            solution[f'x{i}'] = x[i].solution_value()
        #for i in range(n):
        #    solution[f'z{i}'] = z[i].solution_value()
        solutions.append(solution)

        
    return solutions
    
    #
    # Solve the problem
    #status = solver.Solve()
    #
    #
    #
    #if status == pywraplp.Solver.OPTIMAL:
    #    # Print the results
    #    print("Status: Optimal")
    ##    print("Optimal Solution:")
    #    for i in range(m):
    #        print(f"x{i} = {x[i].solution_value()}")
    #    for i in range(n):
    #        print(f"z{i} = {z[i].solution_value()}")
    #else:
    #    print("The problem does not have an optimal solution.")

    


In [3]:
import numpy as np
from scipy.sparse import random

def generate_random_sparse_binary_matrix(rows, cols, density):
    # Generate a random binary matrix with the specified density (percentage of non-zero elements)
    random_matrix = random(rows, cols, density=density, format="csr", dtype=np.float64)
    random_matrix.data = np.array(random_matrix.data >= 0.5, dtype=np.int64)  # Threshold to get binary values
    return random_matrix

In [4]:
# Generate two random binary sparse matrices
rows = 5  # Number of rows in the matrix
cols = 6  # Number of columns in the matrix
density = 0.3  # Density of non-zero elements (percentage)

matrix1 = generate_random_sparse_binary_matrix(rows, cols, density)
matrix2 = generate_random_sparse_binary_matrix(rows, cols, density)

N = generate_random_sparse_binary_matrix(rows, cols, density)
P = generate_random_sparse_binary_matrix(rows, cols, density)

# Convert the sparse matrices to nested lists for OR-Tools
N = matrix1.toarray().tolist()
P = matrix2.toarray().tolist()


In [5]:
solutions = solve_integer_program(rows, cols, N, P, 0.0, 1)

here


In [ ]:
import os 
os.getcwd()

In [ ]:
import pandas as pd
import pickle
#from bds.fpgrowth import preprocess_transaction_list, build_fptree, fpgrowth_on_tree
#from bds.bb import get_ground_truth_count
#from bds.bbSampleTreeApproxCounting import BranchAndBoundNaive
import time 
import random
import numpy as np 

# Load the CSV file using Pandas
data = pd.read_csv('../data/compas_test-binary.csv')

# Print the data
data.head()
#
X = data.to_numpy()
#

In [ ]:
# read labels 
f = open("../data/compas_test.label", "r")
labels = []
for row in f.readlines(): 
    labels.append(list(map(int, row.split(" ")[1:])))

In [ ]:
%cd ..
from bds.fpgrowth import preprocess_transaction_list, build_fptree, fpgrowth_on_tree

In [ ]:
from bds.rule import Rule
from time import time
from gmpy2 import mpz , mpfr
import gmpy2 as gmp


def mpz_set_bits(n: mpz, bits: np.ndarray) -> mpz:
    """return a copy of n and set `bits` to 1 in `n`"""
    for i in bits:
        n = gmp.bit_set(n, int(i))
    return n
def compute_truthtable(X,itemset): 
    ''' poorly optimized function '''
    support_list = [] 
    for i in range(X.shape[0]): 
        if sum([X[i][j] for j in itemset]) == len(itemset): 
            support_list.append(i)
                
    truthtable = mpz_set_bits(mpz(), support_list)
    
    
    return truthtable , support_list
    

In [ ]:
len(rules)

In [ ]:
y = np.array(labels[1], dtype = bool) 
X_bag = [set([j for j, x in enumerate(row) if x]) for row in X]
min_support = 160
ordered_input_data = preprocess_transaction_list(
X_bag, min_support)

tree = build_fptree(ordered_input_data)
frequent_itemsets = set(fpgrowth_on_tree(tree, set(), min_support))

PD = X[y]
ND = X[~y]

N = np.zeros((ND.shape[0], len(frequent_itemsets)))
P = np.zeros((PD.shape[0], len(frequent_itemsets)))

# create map 
positive_counter = 0 
negative_counter = 0 
map_positive = dict() 
map_negative = dict() 

for i in range(X.shape[0]): 
    if y[i]: 
        map_positive[i] = positive_counter 
        positive_counter+=1 
    else: 
        map_negative[i] = negative_counter
        negative_counter+=1 
        


# Now create rules 
rules = [] 
for i,itemset in enumerate(frequent_itemsets):
    tt , support_list = compute_truthtable(X,itemset)
    this_Rule = Rule(id = i+1, name = "rule_" + str(i), cardinality=len(itemset), truthtable=tt)
    # we need the truthtable for this itemset 
    rules.append(this_Rule)
    
    for example in support_list: 
        if y[example]: 
            P[map_positive[example], i] = 1 
        else: 
            N[map_negative[example], i] = 1 
            

                

In [ ]:
m = len(rules)
lmbd = 0.0
ub = 4000000000000000000000000000000000
solutions = solve_integer_program(n, m, N, P, lmbd, ub)

In [ ]:
solutions